In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely import wkt
import matplotlib.pyplot as plt
import rasterio
from rasterio.features import rasterize
import torch
import time

In [2]:
#TODO: parallelisieren / gdal python binding über cli direct call / 24xhöhe von bbox, nur steps nach rechts und dann splitten? / polygonbasiert 


# Bounding box und Geometrie für Sentinel-Abfrage

In [3]:
df_raw = pd.read_csv('bern_landkult.csv')
labels = pd.read_csv('../Level_hierarchy.csv', usecols=['level3', 'LNF_code'])
labels.columns=['level3', 'LNF_CODE']

In [4]:
df_landkult = pd.merge(df_raw, labels, how='left', on='LNF_CODE' )
df_landkult = df_landkult.dropna(subset = ['level3'])

In [5]:
df_landkult['geometry'] = df_landkult['geometry'].apply(wkt.loads)
crs = {'init': 'epsg:2056'}
gdf = gpd.GeoDataFrame(df_landkult, crs=crs, geometry='geometry')
gdf_32632 = gdf.to_crs(32632)
gdf_32632.head()

c:\Users\tmand\.virtualenvs\AgroLuege-2VDmF-75\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,Unnamed: 0,IST_DEF,KULTGRUPPE,KULTGRUT_K,KULTGRUT_1,LNF_CODE,KULTURT_KU,KULTURT__1,NHG,NUTZUNG,...,DARSGRUT_D,DARSGRUT_1,GEOID,HAUPTKAT,HAKAT_HK_D,HAKAT_HK_F,Shape_Leng,Shape_Area,geometry,level3
0,0,0,2041,übrige Nutzfläche Wiese,autres prairies,613,Übrige Dauerwiesen (ohne Weiden),Autres prairies permanentes (sans les pâturages),0,1,...,keine BFF I,aucune SPB I,766613,1,Dauerwiesen,Prairies permanentes,1114.998396,30892.048265,"POLYGON ((399301.942 5185353.379, 399306.541 5...",21.0
1,1,0,2041,übrige Nutzfläche Wiese,autres prairies,613,Übrige Dauerwiesen (ohne Weiden),Autres prairies permanentes (sans les pâturages),0,1,...,keine BFF I,aucune SPB I,309561,1,Dauerwiesen,Prairies permanentes,194.265546,2273.000733,"POLYGON ((395533.888 5202486.154, 395519.846 5...",21.0
2,2,0,2039,übrige Nutzfläche,autres SAU,857,"Hecken-, Feld- und Ufergehölze (mit Pufferstre...","Haies, bosquets champêtres et berges boisées (...",0,1,...,keine BFF I,aucune SPB I,301706,8,"Hecken-, Feld- und Ufergehölze","Haies, bosquets et berges boisées",204.997401,883.631964,"POLYGON ((377044.226 5236483.316, 377041.905 5...",14.0
3,3,0,2048,Grünland,prairie,601,Kunstwiesen (ohne Weiden),Prairies artificielles (sans les pâturages),0,1,...,keine BFF I,aucune SPB I,969184,0,Ackerfläche,Terres cultivées,387.805081,6898.488772,"POLYGON ((390521.806 5184008.833, 390520.286 5...",21.0
4,4,0,2048,Grünland,prairie,601,Kunstwiesen (ohne Weiden),Prairies artificielles (sans les pâturages),0,1,...,keine BFF I,aucune SPB I,950199,0,Ackerfläche,Terres cultivées,213.983203,1756.279697,"POLYGON ((408013.194 5220252.410, 408013.565 5...",21.0


In [6]:
#gdf_32632.plot()

In [7]:
# from sentinelhub import (
#     CRS,
#     BBox,
#     bbox_to_dimensions
# )

Sentinel stellt Funktionen bereit, die mithilfe des Parameters "resolution" die Pixelgrösse angibt. Damit können wir einen durch 240 pixel (=2.4km) teilbaren Ausschnitt wählen, den wir jeweils mit einem request bei Sentinel abfragen können.
Wir werden ein Mosaik aus 23 x 41 Elementen haben für den Kanton Bern.

In [ ]:
x_start = 361630.
y_start = 5140066.
x_end = x_start + 23*2400
y_end = y_start + 41*2400
bbox_to_dimensions(BBox(bbox=(x_start, y_start, x_end, y_end), crs=32632), resolution=10)

In [ ]:
area_box = []
x_start = 361630.
y_start = 5140066.
step= 240
x_end = x_start + 230*step
y_end = y_start + 410*step
x_coordinate = list(np.arange(x_start, x_end, step))
y_coordinate = list(np.arange(y_start, y_end, step))
for i in range(len(x_coordinate)):
    for j in range(len(y_coordinate)):
        area = gdf_32632.cx[x_coordinate[i]:x_coordinate[i]+240, y_coordinate[j]:y_coordinate[j]+240].Shape_Area.sum()/10000
        box_entry = {'x1': x_coordinate[i], 'y1': y_coordinate[j],'x2': x_coordinate[i]+240,'y2': y_coordinate[j]+240, 'area': area}
        area_box.append(box_entry)
area_boxes = pd.DataFrame(area_box)

In [ ]:
plt.scatter(x='x1', y='y1', s='area', data=area_boxes, alpha=0.5, c='g')

In [ ]:
area_boxes.replace(0, np.nan, inplace=True)
print(area_boxes.area.isna().sum())

In [ ]:
bboxes = area_boxes.copy()
bboxes = bboxes.dropna(subset = ['area'])
bboxes.to_csv('../raw_data/BernCrop/bboxes_sentinel_240x240.csv')
bboxes = bboxes.drop(columns=['x2', 'y2', 'area'])

In [15]:
bboxes = pd.read_csv(r'..\raw_data\BernCrop\bboxes_sentinel_24x24.csv',index_col=[0])

In [16]:
bboxes

,x1,y1,x2,y2,area
10,361630.0,5142466.0,361870.0,5142706.0,206.058478
11,361630.0,5142706.0,361870.0,5142946.0,206.058478
12,361630.0,5142946.0,361870.0,5143186.0,206.058478
13,361630.0,5143186.0,361870.0,5143426.0,206.058478
14,361630.0,5143426.0,361870.0,5143666.0,206.058478
...,...,...,...,...,...
94165,416590.0,5206066.0,416830.0,5206306.0,92.922064
94166,416590.0,5206306.0,416830.0,5206546.0,27.224741
94167,416590.0,5206546.0,416830.0,5206786.0,27.224741
94243,416590.0,5224786.0,416830.0,5225026.0,1.695454


### Bounding-Box für ETH

In [ ]:
x_start,x_end, y_start, y_end

In [ ]:
gdf_32632.cx[x_start:x_end, y_start: y_end].plot()

# Rasterize GIS-Data
## Label-Ebene
Die Polygonzüge der einzelnen Landwirtschaftsfelder haben jeweils ein LNF_Code hinterlegt. In einem ersten Schritt wird jedem Polygonzug das Label der 3. Hierarchie (3level) hinterlegt. Die Originaldaten des Kanton Bern werden in das erforderliche Koordinatensystem WGS 84 / UTM zone 32N umgewandelt.
In einem zweiten Schritt werden die Polygon-Label auf Rasterebene umgewandelt, so dass jedes Pixel (10 x 10m) ein Label hat. Das Ergebnis ist eine Liste von numpy-arrays, bei dem jeder array die 2.4 x 2.4km der Sentinel-Abfrage darstellt.

In [11]:
gdf_pair = gdf_32632[['geometry', 'level3']]

In [12]:
test_gdf = gdf_pair.itertuples(index=False, name=None)
test_gdf = list(test_gdf)


In [28]:
bboxes[bboxes.index.isin([i for i in range(0,410)])]

,x1,y1,x2,y2,area
10,361630.0,5142466.0,361870.0,5142706.0,206.058478
11,361630.0,5142706.0,361870.0,5142946.0,206.058478
12,361630.0,5142946.0,361870.0,5143186.0,206.058478
13,361630.0,5143186.0,361870.0,5143426.0,206.058478
14,361630.0,5143426.0,361870.0,5143666.0,206.058478
...,...,...,...,...,...
405,361630.0,5237266.0,361870.0,5237506.0,28.712405
406,361630.0,5237506.0,361870.0,5237746.0,35.447584
407,361630.0,5237746.0,361870.0,5237986.0,44.423441
408,361630.0,5237986.0,361870.0,5238226.0,21.496997


In [ ]:
import time
import rasterio
from rasterio.features import rasterize

def raster(coord, geo_df, field_df):
    """
    coord: DataFrame with x and y coordinates of bounding_boxes
    geo_df: list of tuple with geometry (Polygon) and value pair
    """
    result_list_pixel = []
    result_list_field = []
    #TODO: get bboxes from 0:410 or whatever index
    
    with rasterio.Env():
        for i, row in coord.iterrows():
            x_coord, y_coord = row['x1'], row['y1']
            start_time = time.time()
            result = rasterize(geo_df, out_shape=(240, 240), transform=(10, 0, x_coord, 0, -10, y_coord, 0, 0, 1))
            result_list_pixel.append(result)
            result = rasterize(field_df, out_shape=(240, 240), transform=(10, 0, x_coord, 0, -10, y_coord, 0, 0, 1))
            result_list_field.append(result)
            print(f"End Time: {time.time() - start_time}")
    
    return result_list_pixel,result_list_field


## Feld-Ebene

In [ ]:
gdf_pair_field = gdf_32632[['geometry', 'Unnamed: 0', 'level3']]

In [ ]:
gdf_pair_field.columns=['geometry', 'gt_instance', 'level3']


In [ ]:
gdf_pair_field['gt_instance'] = gdf_pair_field['gt_instance']+1000000
gdf_pair_field.head()

Falls wir die Zuordnung Feld - Label nochmals brauchen, exportieren wir dieses als csv. Dies gäbe uns die Möglichkeit, ein stratified Fold zu machen.

In [ ]:
gdf_pair_field.drop('geometry',axis=1).to_csv(r'gt_instance-level3.csv') 

In [ ]:
gdf_pair_field = gdf_pair_field.drop('level3', axis=1)

In [ ]:
pair_gdf = gdf_pair_field.itertuples(index=False, name=None)
pair_gdf = list(pair_gdf)

In [ ]:
result_pixel,result_field = raster(
    bboxes, test_gdf,pair_gdf)


In [ ]:
plt.imshow(result_pixel[3])
plt.show()
plt.imshow(result_field[3])
plt.show()

In [ ]:
tensor_pixel= torch.Tensor(result_pixel)
print(tensor_pixel.size())
tensor_field= torch.Tensor(result_field)
print(tensor_field.size())

In [ ]:
torch.save(tensor_pixel, '../raw_data/BernCrop/tensor_label_240x240.pt')
torch.save(tensor_field, '../raw_data/BernCrop/tensor_field_240x240.pt')